# 基于TF使用NotMnist数据集训练一个线性模型

In [1]:
import pickle
import numpy as np
import tensorflow as tf

image_size = 28
num_labels = 10

# 封装数据集为对象

In [2]:
class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []

        class Valid:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()
        self.valid = Valid()
        
        pickle_file = '../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存

        def reformat(dataset, labels):
            dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
            labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            return dataset, labels

        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.valid.images = valid_dataset
        self.valid.labels = valid_labels
        self.test.images = test_dataset
        self.test.labels = test_labels

# 创建数据集实例

In [3]:
not_mnist = NotMNIST()


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


# 训练参数设定

In [48]:
# 参数
learning_rate = 0.05  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 40  # 迭代轮数
batch_size = 50  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))


In [49]:
# 构造模型
pred = tf.nn.softmax(tf.matmul(x, W) + b)  # Softmax


In [50]:
# 损失函数：交叉墒
# todo  请使用tf的api定义交叉墒损失函数
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
#cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = pred, labels = y))

# 使用梯度下降算法寻找最优解
#todo 请使用tf函数定义梯度下降优化法方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
#optimizer = tf.train.MomentumOptimizer(learning_rate, 0.05).minimize(cost)
# 测试模型
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# 计算准确度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 初始化所有变量
init = tf.initialize_all_variables()
saver = tf.train.Saver()


Instructions for updating:
Use `tf.global_variables_initializer` instead.


# 开始进行训练，按照迭代次数进行迭代，每个迭代采用valid数据集进行验证

In [51]:
# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
            
        # 打印显示
        if (epoch + 1) % display_step == 0:
            #print(avg_cost)
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost), "valid=", accuracy.eval({x: not_mnist.valid.images, y: not_mnist.valid.labels}))
    print("Optimization Finished!")
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    #saver.save(sess, 'my-model') # 保存模型
    

Epoch: 0001 cost= 1.702906967 valid= 0.8238
Epoch: 0002 cost= 1.652046714 valid= 0.8303
Epoch: 0003 cost= 1.643283358 valid= 0.8336
Epoch: 0004 cost= 1.638477012 valid= 0.8356
Epoch: 0005 cost= 1.635260346 valid= 0.837
Epoch: 0006 cost= 1.632892330 valid= 0.8385
Epoch: 0007 cost= 1.631048215 valid= 0.8394
Epoch: 0008 cost= 1.629555388 valid= 0.8394
Epoch: 0009 cost= 1.628310998 valid= 0.8401
Epoch: 0010 cost= 1.627249352 valid= 0.8412
Epoch: 0011 cost= 1.626326839 valid= 0.8414
Epoch: 0012 cost= 1.625513646 valid= 0.8417
Epoch: 0013 cost= 1.624788705 valid= 0.8415
Epoch: 0014 cost= 1.624136535 valid= 0.8416
Epoch: 0015 cost= 1.623545310 valid= 0.8423
Epoch: 0016 cost= 1.623005720 valid= 0.8425
Epoch: 0017 cost= 1.622510282 valid= 0.8424
Epoch: 0018 cost= 1.622052889 valid= 0.8428
Epoch: 0019 cost= 1.621628516 valid= 0.843
Epoch: 0020 cost= 1.621232973 valid= 0.8432
Epoch: 0021 cost= 1.620862750 valid= 0.8433
Epoch: 0022 cost= 1.620514874 valid= 0.8437
Epoch: 0023 cost= 1.620186830 vali

# 作业一：请尝试切换不同的优化方法，例如使用：
* GradientDescentOptimizer
* AdadeltaOptimizer
* AdagradOptimizer
* MomentumOptimizer
* AdamOptimizer
* FtrlOptimizer
* RMSPropOptimizer

# 观察结果

# 作业二：请进行不同的参数调整，例如minibatch 大小，learning_rate，已经迭代训练的次数，观察结果